In [23]:
# Optimizing a swing trading strategy for cryptocurrency

# This example is adapted from Python For Finance, 2nd ed., Hilpisch, Yves.
# Chapter 15 Trading Strategies, Vectorized Backtesting and
# RelaxedTrader.com - https://relaxedtrader.com/spy-swing-trading-systems-10-day-low/

# A brute force method is used in an attempt to optimize the following values:
# Long Moving Average (LMA): 200 days - detects long upward trend
# Minimum Price (MinPrice): 10 days   - buy on the dip
# Maximum Price (MaxPrice): 5 days    - sell into a rally

# Trading Rules Pseudo Code:

# If Instrument > Long Moving Average then:
#         If Instrument Close < MinPrice then buy
#         ElseIf Instrument Close > MaxPrice then sell

# ** THIS TAKES ABOUT 30 - 60 SEC TO RUN **

from itertools import product
import numpy as np
import pandas as pd

# Load data for brute force optimization

symbol = 'BTC-USD'

tick_data = pd.read_csv(symbol+'.csv', index_col=0, infer_datetime_format=True, parse_dates=True)
tick_data.iloc[:, 0] = pd.to_datetime(tick_data.iloc[:, 0], infer_datetime_format=True, unit='s')
tick_data.set_index('time', inplace=True)
tick_data.sort_values(by='time', inplace=True)
raw = pd.DataFrame(tick_data['close']).dropna()
raw.rename(columns={'close':symbol}, inplace=True)
raw.head

# Brute force optimization
# range(min, max, increment size)
lma = range(20, 280, 20)
min_price = range(1, 21, 2)
max_price = range(1, 21, 2)

results = pd.DataFrame()

for LMA, MinPrice, MaxPrice in product(lma, min_price, max_price):
    data = pd.DataFrame(raw[symbol])
    data.dropna(inplace = True)
    data['Returns'] = np.log(data[symbol]/data[symbol].shift(1))
    data['LMA'] = data[symbol].rolling(LMA).mean()
    data['MinPrice'] = data[symbol].rolling(MinPrice).min()
    data['MaxPrice'] = data[symbol].rolling(MaxPrice).max()
    data.dropna(inplace = True)
    
    # trading rules:
    
    conditions  = [ (data[symbol] >= data['LMA']) & (data[symbol] < data['MinPrice']),\
                   (data[symbol] >= data['LMA']) & (data[symbol] > data['MaxPrice'])]
    
    choices     = [ 1, -1]  # Hold = 1, Sell = -1
    
    data['Position'] = np.select(conditions, choices, default=1) # default: Hold = 1
    
    data['Strategy'] = data['Position'].shift(1)*data['Returns']
    
    data.dropna(inplace = True)
    perf = np.exp(data[['Returns', 'Strategy']].sum())
    results = results.append(pd.DataFrame(
                {'LMA':LMA, 'MinPrice': MinPrice, 'MaxPrice': MaxPrice,
                'Position':data['Position'], 'MARKET': perf['Returns'],
                'STRATEGY': perf['Strategy'], 'OUT': perf['Strategy']-perf['Returns']},
                index=[0]), ignore_index=True)

results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   LMA       1300 non-null   int64  
 1   MinPrice  1300 non-null   int64  
 2   MaxPrice  1300 non-null   int64  
 3   Position  0 non-null      float64
 4   MARKET    1300 non-null   float64
 5   STRATEGY  1300 non-null   float64
 6   OUT       1300 non-null   float64
dtypes: float64(4), int64(3)
memory usage: 71.2 KB


In [24]:
results.sort_values('OUT', ascending=False).head(10)

,LMA,MinPrice,MaxPrice,Position,MARKET,STRATEGY,OUT
0,20,1,1,NaN,4.557128,4.557128,0.0
854,180,11,9,NaN,2.854716,2.854716,0.0
872,180,15,5,NaN,2.854716,2.854716,0.0
871,180,15,3,NaN,2.854716,2.854716,0.0
870,180,15,1,NaN,2.854716,2.854716,0.0
869,180,13,19,NaN,2.854716,2.854716,0.0
868,180,13,17,NaN,2.854716,2.854716,0.0
867,180,13,15,NaN,2.854716,2.854716,0.0
866,180,13,13,NaN,2.854716,2.854716,0.0
865,180,13,11,NaN,2.854716,2.854716,0.0


In [25]:
results.sort_values('OUT', ascending=False).to_csv('results.csv')

In [26]:
data.to_csv('data.csv')

In [18]:
tick_data.head

<bound method NDFrame.head of                  low      high      open     close         volume
time                                                             
2020-03-08   8002.20   8901.37   8901.36   8037.76   16591.914102
2020-03-09   7630.00   8187.03   8037.73   7934.52   25563.614175
2020-03-10   7733.56   8158.25   7934.56   7894.68   17172.471301
2020-03-11   7583.27   7987.97   7894.68   7938.05   13647.865139
2020-03-12   4644.00   7969.45   7938.05   4857.10  113902.203329
...              ...       ...       ...       ...            ...
2020-12-28  26086.19  27477.00  26255.16  27040.36   21265.709350
2020-12-29  25833.73  27396.77  27039.39  27366.35   20668.104338
2020-12-30  27311.73  29018.26  27366.35  28897.42   28181.496815
2020-12-31  28000.00  29321.90  28897.42  28990.08   28813.886911
2021-01-01  28751.82  29083.75  28990.08  29039.66    1204.307665

[300 rows x 5 columns]>